# Malhas e curvaturas

## Confeccionando malhas em $R^2$

In [ ]:
nx = np.linspace(0,10,11)
ny = np.linspace(0,10,11)
h = abs(nx[1] - nx[0])

malha = np.empty((len(nx),len(ny),2))

for i in range(len(nx)):
  for j in range(len(ny)):
    par = np.array([nx[i],ny[j]])
    malha[i,j] = par

print('Dimensões do tensor da malha: ')
print(malha.shape)

plt.style.use('default')
fig, gr = plt.subplots(figsize = (6,4))

for i in range(len(ny)):
  #linhas verticais
  x = malha[i,:,0]
  y = malha[i,:,1]
  
  gr.plot(x, y, 'red')

  #linhas horizontais
  x = malha[:,i,0]
  y = malha[:,i,1]
  
  gr.plot(x, y, 'red')

  #linhas diagonais inferiores
  x = nx[i:]

  gr.plot(x, x - h*i, 'red')

  #linhas diagonais superiores
  x = nx[:-i]
  gr.plot(x, x + h*i, 'red')


'''
malha = np.empty((len(nx)*len(ny),2))
contador = 0

for i in range(len(nx)):
  for j in range(len(ny)):
    par = np.array([nx[i],ny[j]])    
    malha[contador] = par
    contador = contador + 1

print(malha.shape)

fig, ax = plt.subplots()

ax.scatter(malha[:,0],malha[:,1],'red')
for i in range(0,nx*ny/nx):
  ax.plot(malha[i,0], malha[i,1], 'red')

for i in range(nx*ny/nx,nx*ny/nx):
  ax.plot(malha[i,0], malha[i,1], 'red')
'''


Pertubando a malha

In [ ]:
nx = np.linspace(0,0.5,20)
ny = np.linspace(0,0.5,30)


#Pertubações
def f(x):
  return -0.3*np.sin(2*np.pi*2*x/10 - 2*np.pi/4)

def g(x):
  return np.exp(x)

malha = np.empty((len(nx),len(ny),2))

for i in range(len(nx)):
  for j in range(len(ny)):
    par = np.array([nx[i],ny[j]])
    malha[i,j] = par

print('Dimensões do tensor da malha: ')
print(malha.shape)

#Pertubação somente na última linha:
#malha[:,-1,1] = malha[:,-1,1] + f(malha[:,-1,0])

#Pertubação em todas as linhas:
for i in range(len(ny)):
  malha[:,i,1] = malha[:,i,1] + f(malha[:,i,0])

#Pertubação em todas as colunas:
for i in range(len(nx)):
  malha[i,:,0] = malha[i,:,0] + f(malha[i,:,1])


#Plotando
plt.style.use('default')
fig, gr = plt.subplots(figsize = (6,4))

for i in range(len(nx)):
  #linhas verticais e nós
  x = malha[i,:,0]
  y = malha[i,:,1]
  
  gr.plot(x, y, 'red')

for j in range(len(ny)):
  #linhas horizontais
  x = malha[:,j,0]
  y = malha[:,j,1]
  
  gr.plot(x, y, 'red')
  


## Curvatura em $R^2$

Curvatura é a quantidade na qual um objeto geométrico se desvia do plano, portanto a curvatura de um plano
é zero. O cálculo da curvatura é importante em muitas aplicações de engenharia. Como exemplo importante,
cita-se o cálculo da força de tensão superficial em escoamentos bifásicos que apresentam interface maleável.
Curvatura em um ponto n é dado, discretamente, por:

$k_n = \frac{t_{n+1}-t_{n-1}}{\Delta s}$

$t_{n-1}$ e $t_{n+1}$ são vetores que ligão o ponto $n-1$ ao $n$ e $n$ ao $n+1$ e $\Delta s$ é a distância entre $n+1$ e $n-1$.

In [ ]:
nt = np.linspace(0,2*np.pi,1000)
r = 1

#Curvatura real
k_real = 1/r

#Construindo curva
curva = np.empty((nt.size,2))

for i in range(nt.size):
  curva[i] = [r*np.cos(nt[i]), r*np.sin(nt[i])]

#Plotando
plt.style.use('default')
fig, gr = plt.subplots(figsize = (6,6))

x = curva[:,0]
y = curva[:,1]

plt.plot(x, y, 'red')

#Curvatura
K = np.empty(nt.size)

#As primeira e última curvaturas devem ser postas manualmente
#No caso de curvas fechadas, ocorre da maneira abaixo:

t2 = (curva[1] - curva[0])/np.linalg.norm(curva[1] - curva[0])
t1 = (curva[0] - curva[-1])/np.linalg.norm(curva[0] - curva[-1])
#T = (t2 - t1)/np.linalg.norm(curva[1] - curva[-1])
T = (t2 - t1)/np.linalg.norm(t2 - t1)
k = np.linalg.norm(T)
K[0] = k

t2 = (curva[0] - curva[-1])/np.linalg.norm(curva[0] - curva[-1])
t1 = (curva[-1] - curva[-2])/np.linalg.norm(curva[-1] - curva[-2])
#T = (t2 - t1)/np.linalg.norm(curva[0] - curva[-2])
T = (t2 - t1)/np.linalg.norm(t2 - t1)
k = np.linalg.norm(T)
K[-1] = k

for i in range(1,nt.size-1):
  t2 = (curva[i+1] - curva[i])/np.linalg.norm(curva[i+1] - curva[i])
  t1 = (curva[i] - curva[i-1])/np.linalg.norm(curva[i] - curva[i-1])

  #T = (t2 - t1)/np.linalg.norm(curva[i+1] - curva[i-1])  ERRADO, assim a curvatura fica dividida por dois
  T = (t2 - t1)/np.linalg.norm(t2 - t1)

  k = np.linalg.norm(T)

  K[i] = k

print('Curvatura aproximada: ')
print(K[2])
print('Curvatura exata: ')
print(k_real)


## Confeccionando malhas em $R^3$

**Superfície:**

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator

nx = np.linspace(-0.10,0.10,30)
ny = np.linspace(-0.10,0.10,30)

#Pertubações
def f(x):
  return -0.3*np.sin(2*np.pi*2*x/10 - 2*np.pi/4)

def g(x):
  return np.exp(x)

X, Y = np.meshgrid(nx, ny)

#Paraboloide
#Z = (X/2)**2 + (Y/3)**2
#Paraboloide hiperbólico
Z = (X/2)**2 - (Y/3)**2
#Esfera
#Z = np.sqrt(1 - X**2 - Y**2)

# Plot the surface.

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.zaxis.set_major_locator(LinearLocator(10))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


**Volume:**

In [ ]:
from numpy.lib.nanfunctions import nanmax
nx = np.linspace(0,0.1,10)
ny = np.linspace(0,0.4,15)
nz = np.linspace(0,0.10,15)

#Pertubações
def f(x):
  return -0.3*np.sin(2*np.pi*2*x/10 - 2*np.pi/4)

def g(x):
  return np.exp(x)

malha = np.empty((len(nx),len(ny),len(nz),3))

for i in range(len(nx)):
  for j in range(len(ny)):
    for k in range(len(nz)):
      par = np.array([nx[i],ny[j],nz[k]])
      malha[i,j,k] = par

print('Dimensões do tensor da malha: ')
print(malha.shape)

#Pertubação em todas as linhas:

for j in range(len(ny)):
  malha[:,j,:,1] = malha[:,j,:,1] + f(malha[:,j,:,0])

#Pertubação em todas as colunas:
for i in range(len(nx)):
  malha[i,:,:,0] = malha[i,:,:,0] + f(malha[i,:,:,1])


#Plotando
plt.style.use('default')

fig, ax1 = plt.subplots()
ax1 = plt.axes(projection="3d")


for i in range(len(nx)):
  for j in range(len(ny)):
    
    x = malha[i,j,:,0]
    y = malha[i,j,:,1]
    z = malha[i,j,:,2]

    ax1.plot(x, y, z, 'red')

for i in range(len(nx)):
  for k in range(len(nz)):

    x = malha[i,:,k,0]
    y = malha[i,:,k,1]
    z = malha[i,:,k,2]

    ax1.plot(x, y, z, 'red')

for j in range(len(ny)):
  for k in range(len(nz)):

    x = malha[:,j,k,0]
    y = malha[:,j,k,1]
    z = malha[:,j,k,2]

    ax1.plot3D(x, y, z, 'red')

### Curvatura em $R^3$